**weather_data_hour**함수는 아래 github의 내용과 같은 과정에 따라 만들어졌습니다. [[github]](https://github.com/TAEJIN-AHN/Electricity-Load-Prediction/blob/75f9fa88c9d52564ed28664b5ffe8616f9e46856/notebook/weather_data_hour.md)



In [11]:
def weather_data_hour(raw_weather_data_path):
  
  # 패키지 불러오기

  import pandas as pd
  import numpy as np

  weather = pd.read_csv(raw_weather_data_path)

  # 실질적인 결측치 (-9, -99, '-')의 비율이 적은 특성만 사용
  weather = weather[['datetime', 'STN_ID', 'STN', 'PS_hPa', 'PA_hPa', 'IR_nan', 'CA_TOT', 'TA_C',
                    'WS_m/s', 'PV_hPa', 'HM_%', 'WD_16', 'TS_C', 'TD_C', 'VS_nan', 'CA_MID']]

  # 광역시, 특별시 데이터만 사용
  weather = weather[weather['STN'].isin(['서울', '부산', '인천', '대구', '대전', '광주', '울산'])]

  # 동일한 Time index로 통일하기
  STN_list = list(weather['STN'].unique())
  time_list = pd.date_range('2013-1-1 00:00',periods = 90876, freq='h')

  for idx, STN in enumerate(STN_list):
    tmp = pd.DataFrame(columns = ['datetime'])
    tmp['datetime'] = [str(time)[:-3] for time in time_list]
    tmp = pd.merge(tmp, weather[weather['STN']==STN], on = 'datetime', how = 'left')
    if idx == 0:
      aligned_weather = tmp
    else:
      aligned_weather = pd.concat([aligned_weather, tmp])

  # 결측치(nan)나 실질적인 결측치(-9, -99, '-')를 보간법으로 채우기
  STN_list = list(aligned_weather['STN'].unique())
  column_list = list(aligned_weather.columns.values)[3:]

  for idx, STN in enumerate(STN_list):
    # ① 각 STN별로 데이터 나누기
    tmp = aligned_weather[aligned_weather['STN'] == STN]

    # ② 시간순으로 정렬하기
    tmp = tmp.sort_values(by='datetime')

    # ③ -9나 -99인 값을 Null값으로 바꾸기
    for column in column_list:

      if column in ['TA_C', 'TS_C', 'TD_C']:
        # 온도값은 -99가 Null값
        tmp.replace({column : {-99 : np.nan}}, inplace = True)
      else:
        # 온도가 아닌 값은 -9가 Null값
        tmp.replace({column : {-9 : np.nan}}, inplace = True)
                    
    for column in column_list:
      # ④ Pandas Interpolate로 보간법 사용하여 Null값 채우기
      if column in ['WD_16', 'CA_TOT', 'CA_MID', 'VS_nan', 'IR_nan']:
        # 이산형 변수는 최근접 이웃 보간법 사용
        tmp[column] = tmp[column].interpolate(method='nearest')
      else:
        # 연속형 변수는 선형 보간법 사용
        tmp[column] = tmp[column].interpolate(method='linear')

    # ⑤ tmp 합치기
    if idx == 0:
      interpolated_weather = tmp
    else:
      interpolated_weather = pd.concat([interpolated_weather, tmp])

  # 중앙값과 평균으로 전국 기상 대표값 정하기

  column_list = list(interpolated_weather.columns.values)[3:]

  for idx, column in enumerate(column_list):
    # ① 컬럼별로 grouby 수행 (중앙값과 평균값 특성별로 다르게 사용)
    if column in ['WD_16', 'CA_TOT', 'CA_MID', 'VS_nan', 'IR_nan']:
      tmp = pd.DataFrame(interpolated_weather.groupby('datetime', as_index = False)[column].median())
    else:
      tmp = pd.DataFrame(interpolated_weather.groupby('datetime', as_index = False)[column].mean())
    # ② groupby 결과를 인덱스 기준으로 Join
    if idx == 0:
      representative_weather = tmp
    else:
      representative_weather = pd.merge(representative_weather, tmp, on = 'datetime', how = 'inner')
    
  return representative_weather

In [12]:
path = '/content/drive/MyDrive/Colab Notebooks/electricity_load_prediction/weather/weather_20130101_20230515.csv'
representative_weather = weather_data_hour(path)

In [14]:
representative_weather.head()

,datetime,PS_hPa,PA_hPa,IR_nan,CA_TOT,TA_C,WS_m/s,PV_hPa,HM_%,WD_16,TS_C,TD_C,VS_nan,CA_MID
0,2013-01-01 00:00,1021.757143,1013.171429,3.0,0.0,-5.871429,1.642857,2.800000,71.142857,5.0,-2.000000,-10.485714,2000.0,0.0
1,2013-01-01 01:00,1021.657143,1013.071429,3.0,0.0,-5.971429,1.457143,2.871429,72.857143,11.0,-2.128571,-10.228571,2000.0,0.0
2,2013-01-01 02:00,1021.728571,1013.157143,3.0,6.0,-6.057143,1.642857,2.928571,74.285714,7.0,-2.200000,-10.028571,1800.0,5.0
3,2013-01-01 03:00,1021.600000,1013.014286,3.0,6.0,-6.071429,1.614286,2.942857,75.857143,7.0,-2.128571,-9.857143,1800.0,5.0
4,2013-01-01 04:00,1020.785714,1012.200000,3.0,2.0,-6.271429,1.728571,2.914286,75.714286,9.0,-2.342857,-10.042857,1800.0,2.0
